<a href="https://colab.research.google.com/github/MiguelEuripedes/embedded_AI/blob/main/Projects/first_image_classifier/mlp_classifier/train_MLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Your First Image Classifier: Using MLP to Classify Images

## Train

The purpose of this dataset is to correctly classify an image as containing a dog, cat, or panda. Containing only 3,000 images, the Animals dataset is meant to be another introductory dataset that we can quickly train a Multilayer Perceptron (MLP) model and obtain results that can be compared with the previously trained KNN model and the future CNN model.

Let's take the following steps:

1. Encoding target variable
2. Training the MLP model
3. Export the model and the encoder object

### Step 01: Setup

Start out by installing the experiment tracking library and setting up your free W&B account:

* **pip install wandb** – Install the W&B library
* **import wandb** – Import the wandb library
* **wandb login** – Login to your W&B account so you can log all your metrics in one place

In [ ]:
!pip install wandb -qU

     |████████████████████████████████| 1.9 MB 4.9 MB/s 
     |████████████████████████████████| 182 kB 73.6 MB/s 
     |████████████████████████████████| 162 kB 71.9 MB/s 
     |████████████████████████████████| 63 kB 2.0 MB/s 
     |████████████████████████████████| 162 kB 69.8 MB/s 
     |████████████████████████████████| 158 kB 74.6 MB/s 
     |████████████████████████████████| 157 kB 58.7 MB/s 
     |████████████████████████████████| 157 kB 65.5 MB/s 
     |████████████████████████████████| 157 kB 73.7 MB/s 
     |████████████████████████████████| 157 kB 62.8 MB/s 
     |████████████████████████████████| 157 kB 78.9 MB/s 
     |████████████████████████████████| 157 kB 73.0 MB/s 
     |████████████████████████████████| 157 kB 80.8 MB/s 
     |████████████████████████████████| 156 kB 68.9 MB/s 


In [ ]:
import wandb
wandb.login()

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 

··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
!pip install codecarbon

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 174 kB 4.6 MB/s 
     |████████████████████████████████| 46 kB 5.1 MB/s 
     |████████████████████████████████| 66 kB 5.2 MB/s 
     |████████████████████████████████| 99 kB 4.7 MB/s 
  Created wheel for py-cpuinfo: filename=py_cpuinfo-8.0.0-py3-none-any.whl size=22257 sha256=ab1e6f1bea100830345d3bd7f0d0c927b3ae0f57ca8c73ce6daf8a68271d6cee
  Stored in directory: /root/.cache/pip/wheels/d2/f1/1f/041add21dc9c4220157f1bd2bd6afe1f1a49524c3396b94401
Successfully built py-cpuinfo


**Import the packages**

In [ ]:
from imutils import paths
import logging
import os
import cv2
import numpy as np
import joblib
from sklearn.metrics import fbeta_score, precision_score, recall_score, accuracy_score
from codecarbon import EmissionsTracker
from sklearn.preprocessing import LabelEncoder
from sklearn.neural_network import MLPClassifier

Configure logging reference for a logging object

In [ ]:
logger = logging.getLogger()

Set level of logging:

In [ ]:
logger.setLevel(logging.INFO)

Create handlers:

In [ ]:
c_handler = logging.StreamHandler()
c_format = logging.Formatter(fmt="%(asctime)s %(message)s",datefmt='%d-%m-%Y %H:%M:%S')
c_handler.setFormatter(c_format)

Add handler to the logger:

In [ ]:
logger.handlers[0] = c_handler

### Step 02: Basic configuration and download artifacts

With the same strategy as before, we'll replace our argument parsing code with hard coded arguments and values

In [ ]:
args = {
  "project_name": "mlp_classifier",
  "train_feature_artifact": "train_x:latest",
  "train_target_artifact": "train_y:latest",
  "val_feature_artifact": "val_x:latest",
  "val_target_artifact": "val_y:latest",
  "encoder": "target_encoder",
  "inference_model": "model"
}

Since we did data segregation before, lets use it here. Lets start by downloading the data from W&B that was created in our previous notebook.

First lets connect properly with W&B

In [ ]:
run = wandb.init(entity="euripedes",project=args["project_name"], job_type="Train")

wandb: Currently logged in as: euripedes. Use `wandb login --relogin` to force relogin


Now lets download the data

In [ ]:
# train x
train_x_artifact = run.use_artifact(args["train_feature_artifact"])
train_x_path = train_x_artifact.file()

# train y
train_y_artifact = run.use_artifact(args["train_target_artifact"])
train_y_path = train_y_artifact.file()

# validation x
val_x_artifact = run.use_artifact(args["val_feature_artifact"])
val_x_path = val_x_artifact.file()

# validation y
val_y_artifact = run.use_artifact(args["val_target_artifact"])
val_y_path = val_y_artifact.file()

# unpacking the artifacts
train_x = joblib.load(train_x_path)
train_y = joblib.load(train_y_path)
val_x = joblib.load(val_x_path)
val_y = joblib.load(val_y_path)

Now we can make sure the data is correct:

In [ ]:
logger.info("Train x: {}".format(train_x.shape))
logger.info("Train y: {}".format(train_y.shape))
logger.info("Validation x: {}".format(val_x.shape))
logger.info("Validation y: {}".format(val_y.shape))

16-10-2022 16:31:39 Train x: (1687, 3072)
16-10-2022 16:31:39 Train y: (1687,)
16-10-2022 16:31:39 Validation x: (563, 3072)
16-10-2022 16:31:39 Validation y: (563,)


### Step 03: Encoder the target variable 

We'll be encoding the labels as intergers

In [ ]:
le = LabelEncoder()
train_y = le.fit_transform(train_y)
val_y = le.transform(val_y)

## Hyperparameter tuning with sweep:

Lets first configure the sweep with the desired parameters for our model

In [ ]:
sweep_config = {
  'method':'grid',
  'metric':{
      'name':'accuracy',
      'goal':'maximize'
  },
  'parameters': {
      'hidden_layer_sizes': {
          'values':[(128,128), (128,128,128), (256,256), (256,256,256), (512,512), (512,512,512)]
      },
      'activation':{
          'values': ['relu']  
      },
      'learning_rate': {
            'values': ['constant', 'adaptive'],  
      },
      'max_iter':{
          'values': [1000]
      },
      'solver':{
          'values': ['sgd', 'adam']
      }
  }
}

Now we comunicate to W&B with the sweep config created

In [ ]:
sweep_id = wandb.sweep(sweep_config, entity="euripedes", project=args["project_name"])

Create sweep with ID: q9k4r2k2
Sweep URL: https://wandb.ai/euripedes/mlp_classifier/sweeps/q9k4r2k2


### Step 04: Model definition and Training

Train the MLP classifier on the raw pixel intensities using Keras and the Sweep configuration. In addition we'll use the codecarbon tracker in order to observe the CO2 emission during training.

In [ ]:
def train():
    with wandb.init() as run:

      # create codecarbon tracker
      # codecarbon is too much verbose, change the log level for more info
      tracker = EmissionsTracker(log_level="critical")
      tracker.start()

      # initialize the optimizer and model
      print("[INFO] compiling model...")
      model = MLPClassifier(hidden_layer_sizes=run.config.hidden_layer_sizes,
                          activation=run.config.activation,
                          solver=run.config.solver,
                          learning_rate=run.config.learning_rate,
                          max_iter=run.config.max_iter
                          )

      # train the model
      print("[INFO] training network...")
      model.fit(train_x, train_y)

      # get co2 emissions from tracker
      # "CO2 emission (in Kg)"
      emissions = tracker.stop()

      # make predictions
      predictions = model.predict(val_x)

      # Evaluation Metrics
      logger.info("Validation Evaluation metrics")
      fbeta = fbeta_score(val_y, predictions, beta=1, zero_division=1, average='weighted')
      precision = precision_score(val_y, predictions, zero_division=1, average='weighted')
      recall = recall_score(val_y, predictions, zero_division=1, average='weighted')
      acc = accuracy_score(val_y, predictions)

      logger.info("Validation Accuracy: {}".format(acc))
      logger.info("Validation Precision: {}".format(precision))
      logger.info("Validation Recall: {}".format(recall))
      logger.info("Validation F1: {}".format(fbeta))

      run.summary["Acc"] = acc
      run.summary["Precision"] = precision
      run.summary["Recall"] = recall
      run.summary["F1"] = fbeta
      
      # energy unit is kWh
      run.summary["Energy_Consumed"] = tracker.final_emissions_data.energy_consumed
      run.summary["Energy_RAM"] = tracker.final_emissions_data.ram_energy
      run.summary["Energy_GPU"] = tracker.final_emissions_data.gpu_energy
      run.summary["Energy_CPU"] = tracker.final_emissions_data.cpu_energy
      # kg
      run.summary["CO2_Emissions"] = tracker.final_emissions_data.emissions

      run.finish()

In [ ]:
wandb.agent(sweep_id, function=train)

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.
wandb: Agent Starting Run: u1436alt with config:
wandb: 	activation: relu
wandb: 	hidden_layer_sizes: [128, 128]
wandb: 	learning_rate: constant
wandb: 	max_iter: 1000
wandb: 	solver: sgd


[INFO] compiling model...
[INFO] training network...


16-10-2022 15:55:23 Validation Evaluation metrics
16-10-2022 15:55:24 Validation Accuracy: 0.30728241563055064
16-10-2022 15:55:24 Validation Precision: 0.7871400673251958
16-10-2022 15:55:24 Validation Recall: 0.30728241563055064
16-10-2022 15:55:24 Validation F1: 0.1444561356089273


Acc,0.30728
CO2_Emissions,2e-05
Energy_CPU,2e-05
Energy_Consumed,3e-05
Energy_GPU,0.0
Energy_RAM,0.0
F1,0.14446
Precision,0.78714
Recall,0.30728


wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: us32h4us with config:
wandb: 	activation: relu
wandb: 	hidden_layer_sizes: [128, 128]
wandb: 	learning_rate: constant
wandb: 	max_iter: 1000
wandb: 	solver: adam


...

16-10-2022 16:26:08 Validation Evaluation metrics
16-10-2022 16:26:08 Validation Accuracy: 0.5896980461811723
16-10-2022 16:26:08 Validation Precision: 0.6131761780377958
16-10-2022 16:26:08 Validation Recall: 0.5896980461811723
16-10-2022 16:26:08 Validation F1: 0.5975266696157664


Acc,0.5897
CO2_Emissions,0.00095
Energy_CPU,0.00113
Energy_Consumed,0.00151
Energy_GPU,0.00025
Energy_RAM,0.00013
F1,0.59753
Precision,0.61318
Recall,0.5897


wandb: Sweep Agent: Waiting for job.
wandb: Sweep Agent: Exiting.


Error in callback <function _WandbInit._pause_backend at 0x7ff32ee679e0> (for post_run_cell):


BrokenPipeError: ignored

### Dumping the model and encoder artifacts to disk and upload to W&B

Now with the results, lets chose the best model to save at W&B. We'll use the same hyperparams that gave us the best result. Can send it to W&B using joblib

In [ ]:
model = MLPClassifier(hidden_layer_sizes=(256, 256, 256),
                          activation='relu',
                          max_iter=1000,
                          learning_rate = 'constant',
                          solver='adam')

model.fit(train_x, train_y)

MLPClassifier(hidden_layer_sizes=(256, 256, 256), max_iter=1000)

Now, with the model lest save it to W&B

In [ ]:
logger.info("Dumping the model and encoder artifacts to the disk")

# Save the artifacts
joblib.dump(le, args["encoder"])
joblib.dump(model, args["inference_model"])

16-10-2022 16:33:02 Dumping the model and encoder artifacts to the disk


['model']

**Encoder artifact**

In [ ]:
# encoder artifact
artifact = wandb.Artifact(args["encoder"],
                          type="INFERENCE_MODEL",
                          description="A json file representing the target encoder"
                          )

logger.info("Logging the target encoder artifact")
artifact.add_file(args["encoder"])
run.log_artifact(artifact)

16-10-2022 16:33:04 Logging the target encoder artifact


**Inference model artifact**

In [ ]:
# inference model artifact
artifact = wandb.Artifact(args["inference_model"],
                          type="INFERENCE_MODEL",
                          description="A json file representing the inference model"
                          )

logger.info("Logging the inference model artifact")
artifact.add_file(args["inference_model"])
run.log_artifact(artifact)

16-10-2022 16:33:07 Logging the inference model artifact


In [ ]:
run.finish()